In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

Importing the dataset

In [3]:
movies = pd.read_csv("movies.dat", sep="::", encoding = "latin-1")
ratings = pd.read_csv("ratings.dat", sep="::",  encoding = "latin-1")
users = pd.read_csv("users.dat", sep="::", encoding = "latin-1")

<ipython-input-3-853bd1d057fe>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv("movies.dat", sep="::", encoding = "latin-1")
<ipython-input-3-853bd1d057fe>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("ratings.dat", sep="::",  encoding = "latin-1")
<ipython-input-3-853bd1d057fe>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv("users.dat", sep="

In [4]:
movies.head()

,1,Toy Story (1995),Animation|Children's|Comedy
0,2,Jumanji (1995),Adventure|Children's|Fantasy
1,3,Grumpier Old Men (1995),Comedy|Romance
2,4,Waiting to Exhale (1995),Comedy|Drama
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller


In [7]:
ratings.head()
# first column is user id, 2nd column is movie id, 3rd column is rating, 4th columnn is time

,1,1193,5,978300760
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268


In [8]:
users.head()
# 1st column is id, 2nd is gender, 3rd is age, 4th is job description, 5th is time

,1,F,1.1,10,48067
0,2,M,56,16,70072
1,3,M,25,15,55117
2,4,M,45,7,02460
3,5,M,25,20,55455
4,6,F,50,9,55117


Importing testing and training data

In [9]:
train_set = pd.read_csv("u3.base", delimiter='\t')
test_set = pd.read_csv("u3.test", delimiter='\t')

In [10]:
train_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,6,5,887431973
4,1,7,4,875071561


In [11]:
# Conveting to numpy array
train_set = np.array(train_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')

In [12]:
train_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [14]:
# Getting the number of users and movies
no_users = int(max(max(train_set[:, 0]), max(test_set[:,0])))
no_movies = int(max(max(train_set[:, 1]), max(test_set[:,1])))
print("Number of users:", no_users)
print("Number of movies:", no_movies)

Number of users: 943
Number of movies: 1682


Converting the data into array with users in lines and movies in columns

In [15]:
def convert(data):
  new_data = []
  for id in range(1, no_users +1):
    movie_id = data[:, 1][data[:, 0] == id]
    rating_id = data[:, 2][data[:, 0] == id]
    ratings = np.zeros(no_movies)
    ratings[movie_id -1] = rating_id
    new_data.append(list(ratings))
  return new_data

In [16]:
train_set = convert(train_set)
test_set = convert(test_set)

In [19]:
len(train_set)

943

Coverting data into torch tensors

In [20]:
train_set = torch.FloatTensor(train_set)
test_set = torch.FloatTensor(test_set)

Converting ratings into binary ratings 1 (liked) or 0 (not liked)

Our recommendation system is based on likes and dislikes

In [21]:
train_set[train_set == 0] = -1 # -1 indicates default value
train_set[train_set == 1] = 0
train_set[train_set == 2] = 0
train_set[train_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

Creating our neural architecture

In [34]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)   #100*1682
        self.a = torch.randn(1, nh)    #1*100
        self.b = torch.randn(1, nv)    #1*1682
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())    #100*1682 * 1682*100 = 100*100
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)     #100*100
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)    #100*100 * 100*1682 = 100*1682
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)     #100*1682
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0)- torch.mm(vk.t(), phk)).t()    #100*1682
                             #!= 1682*100 * 100*100 - 1682*100 * 100*100 = 1682*100
        self.b += torch.sum((v0-vk), 0)
        self.a += torch.sum((ph0-phk), 0)

In [35]:
nv = len(train_set[0])
nh = 100
batch_size = 128
rbm = RBM(nv, nh)
print(rbm.W.size())

torch.Size([100, 1682])


Training the model

In [36]:
epochs = 10
for epoch in range(1, epochs + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, no_users - batch_size, batch_size):
        vk = train_set[id_user:id_user + batch_size]
        v0 = train_set[id_user:id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))

epoch: 1 loss: tensor(0.3348)
epoch: 2 loss: tensor(0.2443)
epoch: 3 loss: tensor(0.2463)
epoch: 4 loss: tensor(0.2532)
epoch: 5 loss: tensor(0.2510)
epoch: 6 loss: tensor(0.2514)
epoch: 7 loss: tensor(0.2526)
epoch: 8 loss: tensor(0.2509)
epoch: 9 loss: tensor(0.2507)
epoch: 10 loss: tensor(0.2547)


Testing the RBM

In [39]:
test_loss = 0
s = 0.
for id_user in range(no_users):
    v = train_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2632)
